# Download Active Listings
Etsy API limits the offset to 12,000 listings, so that is the maximum number of recently updated listings that can be accessed. (Requested 100 at a time because each request is limited to 100 records)

In [1]:
import requests
import pandas as pd
import json

Get all listings that match keywords "stoneware" and "handmade"

In [2]:
#GetListings URL and Parameters for the API
url = "https://openapi.etsy.com/v3/application/listings/active"
key = "r8ymjsnfohuidr1wev7e4cfg"
keyword = "stoneware,handmade"

In [3]:
#initialize the dataframes
dataframe = pd.DataFrame()
df = pd.DataFrame()

### 3/24 I changed this loop a little and need to re-run it

In [4]:
#loop that increases the offset by 100 each time due to the API limitation of downloading a maximum of 100 listings at a time
limit = 100
offset = 0
for i in range (1,121):
    parameters = {"client_id":key, "limit":limit, "offset":offset, "keywords":keyword}
    download = requests.get(url, params=parameters)
    data = download.json()
    if "error" in data.keys():
        print("Maximum Offset Reached")
    else:
        dataframe = pd.json_normalize(data['results'])
        df = df.append(dataframe, ignore_index = True)
    offset = offset + limit
    i = i+1
    

In [5]:
#confirm that there are 12,000 rows
len(df)

12000

In [6]:
#write the data to a csv file
#df.to_csv('12000listingsStonewareHandmade032222.csv')

## Download Shop Info for all the Etsy listings in the data



In [7]:
#NOTE: DELETE LATER You dont need this if you are doing downloading the 12,000 listings and the shop info at the same time
#df = pd.read_csv('12000listingsStonewareHandmade032222.csv')

In [27]:
#Get all the unique shop IDs
shop_ids = df["shop_id"].tolist()
shop_list = list(set(shop_ids))

In [30]:
? shop_list

In [31]:
#initialize the dataframes
dataframeShop = pd.DataFrame()
dfShop = pd.DataFrame()

# HALP SOMETHING IS UP WITH THIS LOOP

In [24]:
#loop that queries the API for 
limit = 100

for j in range(0, len(shop_list)):
    shop_id = str(shop_list[j])
    url = "https://openapi.etsy.com/v3/application/shops/" + shop_id
    parameters = {"client_id":key, "limit":limit}
    download = requests.get(url, params=parameters)
    dataShops = download.json()
    dataframeShop = pd.json_normalize(dataShops)
    dfShop = df.append(dataframeShop, ignore_index = True)

In [26]:
dfShop

,listing_id,user_id,shop_id,title,description,state,creation_timestamp,ending_timestamp,original_creation_timestamp,last_modified_timestamp,...,policy_shipping,policy_refunds,policy_additional,policy_seller_info,policy_update_date,policy_has_private_receipt_info,has_unstructured_policies,policy_privacy,review_average,review_count
0,1.835151e+08,10553830,5960357,Pottery Spoon Rest in Patina Glaze. Wheel thro...,"Simple, functional, and beautiful. Keep your c...",active,1.648668e+09,1.659208e+09,1.395444e+09,1.648668e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.192431e+09,96218501,13557513,Vintage Lidded Stoneware Earthenware Container...,Vintage handmade lidded container. Extremely w...,active,1.648667e+09,1.659208e+09,1.648667e+09,1.648667e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.711739e+07,7696932,5598733,Spoon rest hand thrown pottery ceramic stonewa...,Place this spoon rest next to your stove or ho...,active,1.648667e+09,1.659208e+09,1.322323e+09,1.648667e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.206386e+09,5338835,5203882,3 Tree of Life Heart Pendants - Ceramic Heart ...,3 of my stoneware handmade heart beads. 2 Beau...,active,1.648667e+09,1.659208e+09,1.648667e+09,1.648667e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.116819e+09,48256956,18365601,"Plate / Shallow Bowl, 10 inches, cherry design...",Handmade of chocolate stoneware clay with a de...,active,1.648667e+09,1.659208e+09,1.638215e+09,1.648667e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11996,1.181946e+09,7748500,5606566,Vintage handmade soap dispenser wheel thrown b...,Vintage signed handmade drippy glaze stoneware...,active,1.647187e+09,1.657728e+09,1.647187e+09,1.647192e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11997,5.973933e+08,35220972,8328465,Stoneware Cup without handle.Natural Plant Orn...,Beautiful cup for Nature lovers. Cup with mini...,active,1.647187e+09,1.657727e+09,1.519933e+09,1.647187e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11998,5.963359e+08,35220972,8328465,Blue Stoneware mug/ coffe mug/ blue ceamic cup...,This mug is a wheel thrown. It&#39;s made of w...,active,1.647187e+09,1.657727e+09,1.519565e+09,1.647187e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11999,9.642722e+08,69964264,11438447,"Handmade Ceramic Mug by Rowena Williams, White...","Stoneware mug, hand thrown, white glaze on top...",active,1.647187e+09,1.657727e+09,1.615390e+09,1.647187e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge the data

In [12]:
#create a dataframe with just the columns I'm interested in for the listings data
listings = df[['listing_id', 'shop_id', 'title',
       'description', 'num_favorers',
       'listing_type', 'tags', 'materials',
       'style','taxonomy_id','price.amount',
       'price.divisor', 'price.currency_code','creation_timestamp', 'ending_timestamp',
       'original_creation_timestamp','quantity']]

In [13]:
#create a dataframe with just the columns I'm interested in for the shops data
shops = dfShop[['shop_id', 'shop_name', 'create_date', 'title',
       'announcement', 'currency_code', 'is_vacation', 'vacation_message',
       'sale_message',
       'listing_active_count',
       'accepts_custom_requests', 'url',
       'num_favorers', 'languages', 'is_shop_us_based', 'transaction_sold_count',
       'shipping_from_country_iso', 'shop_location_country_iso',
       'review_average', 'review_count']]

In [14]:
#merge the two datasets so that the shop information is added to each listing
data = listings.merge(shops, on="shop_id")

In [16]:
# save the merged data
#change the date each time
#maybe automate the filename with the correct date someday

#data.to_csv('mergedData033022.csv')

In [17]:
data

,listing_id,shop_id,title_x,description,num_favorers_x,listing_type,tags,materials,style,taxonomy_id,...,accepts_custom_requests,url,num_favorers_y,languages,is_shop_us_based,transaction_sold_count,shipping_from_country_iso,shop_location_country_iso,review_average,review_count
0,183515097,5960357,Pottery Spoon Rest in Patina Glaze. Wheel thro...,"Simple, functional, and beautiful. Keep your c...",517,physical,"[little e pottery, spoonrest, spoon rest, tea ...",[Ceramic],[],1040,...,NaN,https://www.etsy.com/listing/183515097/pottery...,517,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,183515097,5960357,Pottery Spoon Rest in Patina Glaze. Wheel thro...,"Simple, functional, and beautiful. Keep your c...",517,physical,"[little e pottery, spoonrest, spoon rest, tea ...",[Ceramic],[],1040,...,NaN,https://www.etsy.com/listing/1098753597/wheel-...,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,183515097,5960357,Pottery Spoon Rest in Patina Glaze. Wheel thro...,"Simple, functional, and beautiful. Keep your c...",517,physical,"[little e pottery, spoonrest, spoon rest, tea ...",[Ceramic],[],1040,...,NaN,https://www.etsy.com/listing/1173966478/wheel-...,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,183515097,5960357,Pottery Spoon Rest in Patina Glaze. Wheel thro...,"Simple, functional, and beautiful. Keep your c...",517,physical,"[little e pottery, spoonrest, spoon rest, tea ...",[Ceramic],[],1040,...,NaN,https://www.etsy.com/listing/1124381023/wheel-...,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183515097,5960357,Pottery Spoon Rest in Patina Glaze. Wheel thro...,"Simple, functional, and beautiful. Keep your c...",517,physical,"[little e pottery, spoonrest, spoon rest, tea ...",[Ceramic],[],1040,...,NaN,https://www.etsy.com/listing/996560085/wheel-t...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244288,1004557501,26164906,Handmade bonsai pot #17,Open diameter: 3.51” \nHeight: 1.6”\n\nA tenmo...,0,physical,"[pot, ceramics, pottery, bonsai, stoneware, po...","[ceramics, stoneware, clay, glaze]",[],6096,...,NaN,https://www.etsy.com/listing/1002684898/small-...,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244289,1004557501,26164906,Handmade bonsai pot #17,Open diameter: 3.51” \nHeight: 1.6”\n\nA tenmo...,0,physical,"[pot, ceramics, pottery, bonsai, stoneware, po...","[ceramics, stoneware, clay, glaze]",[],6096,...,NaN,https://www.etsy.com/listing/1004557501/handma...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244290,1180383448,6670644,"Studio Pottery Vase & Candle Holder, ALFADOM D...",Thank you for checking out this great set of T...,14,physical,"[1980s 80s 1970s 70s, hand made handmade, stud...","[earthenware, Semi matte drip glaze, Original ...","[Mid Century, Modern]",1026,...,NaN,https://www.etsy.com/listing/1180383448/studio...,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244291,1035873424,16215126,Set of 2 | Salt Glazed Stoneware Vessels | Vin...,Pair of 2 vintage salt glazed stoneware vessel...,12,physical,"[narrow neck pottery, shelf decor, flower vase...",[],[],1026,...,NaN,https://www.etsy.com/listing/1035873424/set-of...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
